# TP2-3

In [2]:
from collections import namedtuple
import os

In [3]:

def xor(a,b):
    l = len(a) if len(a)<len(b) else len(b)
    r = bytearray(l)
    for i in range(0,l):
        r[i] = a[i] ^^ b[i]
    return r

def hash_to_range(n, s, hashfcn):

    # hashlen: the number of octets comprising the output of hashfcn
    hashlen = len(hashfcn(b''))

    v_i = 0
    h_i = b'\x00' * hashlen

    for i in range(1, 3):
        
        # concatenate h_(i-1) and s to form t_i
        t_i = h_i + s

        # hash of t_i
        h_i = hashfcn(t_i)

        # convert h_i to integer a_i
        a_i = int.from_bytes(h_i, 'big')

        # compute v_i
        v_i = 256 ** hashlen * v_i + a_i

    v = v_i % n

    return v

def sha1(v):
    return hashlib.sha1(v).digest()

def sha224(v):
    return hashlib.sha3_224(v).digest()

def sha256(v):
    return hashlib.sha3_256(v).digest()

def sha384(v):
    return hashlib.sha3_384(v).digest()

def sha512(v):
    return hashlib.sha3_512(v).digest()

BFSecParam = namedtuple('SecParam', 'bq bp hashfn')

class BFSecParams:
    n1024 = BFSecParam(160,512,sha1)
    n2048 = BFSecParam(224,1024,sha224)
    n3072 = BFSecParam(256,1536,sha256)
    n7680 = BFSecParam(384,3840,sha384)
    n15360 = BFSecParam(512,7680,sha512)


class BF:
    
    def __init__(self, params):
        bq = params.bq
        bp = params.bp
        
        q = random_prime(2^bq-1,lbound=2^(bq-1))
        
        t = q*3*2^(bp - bq)
        while not is_prime(t-1):
            t = t << 1
        
        p = t - 1
        if not is_prime(p):
            raise Exception("not prime")
        
        # Aneis e Corpos
        Fp     = GF(p)                  # corpo primo com "p" elementos
        R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(p^2, modulus=f)   
        
        # Curvas Elipticas supersingulares em Sagemath
        
        # a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
        E2 = EllipticCurve(Fp2, [0,1])
        
        # ponto arbitrário  de ordem "q" em E2        
        cofac = (p + 1)//q
        G = cofac * E2.random_point()
    
        s = hash_to_range(os.urandom(params.bq//8), q-1, params.hashfn)

        G_pub = s*G
        
        self.s = s
        self.public = G_pub
        self.G = G
        self.params = params
        self.E2 = E2
        self.Fp2 = Fp2
        self.Fp = Fp

    def _ID(self, id):
        return int.from_bytes(self.params.hashfn(id),'little') * self.G

    def _Zr(self):
        return hash_to_range(os.urandom(self.params.bq//8),self.params.q-1,self.params.hashfn)

    def _H(self,v):
        return hash_to_range(v,self.params.q-1,self.params.hashfn)

    def _trace(self,x):       # função linear que mapeia Fp2  em  Fp
        return x + x^self.p

    def _phi(self,P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
        (x,y) = P.xy()
        return self.E2(z*x,y)
    
    def _tateX(self,P,Q,l=1):      # o emparelhamento de Tate generalizado
        return P.tate_pairing(phi(Q), self.q, 2)^l
    
    def _ddhp(self,P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
        return tateX(P,Q) == tateX(R,G)
    
    
    def key_extract(self, id):
        d = self._ID(id)
        return self.s*d
        
    def encrypt(self, id, x):
        def _in(self,id,x):
            d = self.ID(id)
            v = self._Zr()
            a = self._H(xor(int(v).to_bytes(self.params.bq//8,'little'),x))
            u = self._tateX(self.public,d,a)
            return (x,v,a,u)

        def _out(self,x,v,a,u):
            alpha = a*self.G
            vl = xor(int(v).to_bytes(self.params.bq//8,'little'),self._trace(u))
            vb = v.to_bytes(self.params.bq//8,'little')
            xl = xor(x,int(self._H(vb)).to_bytes(self.params.bq//8,'little'))
            return (alpha,vl,xl)
            
        x,v,a,u = _in(self,id,x)
        return _out(self,x,v,a,u)

    def decrypt(self,key,c):
        def _in(self,key,alpha,vl,xl):
            u = self._tateX(alpha,key,1)
            v = vl^^self._trace(u)
            vb = int(v).to_bytes(self.params.bq//8,'little')
            vh = self._H(vb)
            x = xor(xl,)
        def _out(alpha,v,x):

In [4]:
bf = BF(BFSecParams.n2048)